### 1 Generacja liczby pierwszej, test na generator, metoda wyznaczająca wartości potęg kolejnych generatorów, zastosowanie w PRNG

In [6]:
import random

def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True

def primerange(start, end):
    # liczby pierwsze w przedziale start:end
    primes = []
    for i in range(start, end+1):
        if is_prime(i):
            primes.append(i)
    return primes

def random_prime_in_range(start, end):
    # losowa liczba pierwsza z przedziału
    primes = primerange(start, end)
    return random.choice(primes)


def is_generator(g, p):
    # Sprawdzamy, czy g to generator grupy p
    if not is_prime(p):
        raise ValueError("p musi byc liczba pierwsza")

    #Musimy sprawdźić czy kolejne wartości g**k % p generują cały zbiór {1, 2, ..., p-1}
    values = set(pow(g, i, p) for i in range(1, p))
    return len(values) == p - 1

def generate_powers(g, p):
    # Generuje ciąg potęg generatora g modulo p
    return [pow(g, i, p) for i in range(1, p)]

def powers_all_generators(p):
    # Wypisz wartości potęg dla generatorów
    for i in range(1, p):
        if is_generator(i, p):
            print(f"Generator: {i}, powers: {generate_powers(i, p)}")

***Czy indeks będzie generatorem 16-bitowej liczby pierwszej p?*** 

In [37]:
p = random_prime_in_range(0, 2**17-1)
index = 411659
# Żeby problem miał sens weźmy wartość index % p
index %= p
print(f"p: {p}")
print(f"Indeks: {index} {"jest" if is_generator(index, p) else "nie jest"} generatorem grupy {p}")

p: 127217
Indeks: 30008 jest generatorem grupy 127217


***Generacja ciągu potęg dla kolejnych generatorów***

In [38]:
powers_all_generators(11)

Generator: 2, powers: [2, 4, 8, 5, 10, 9, 7, 3, 6, 1]
Generator: 6, powers: [6, 3, 7, 9, 10, 5, 8, 4, 2, 1]
Generator: 7, powers: [7, 5, 2, 3, 10, 4, 6, 9, 8, 1]
Generator: 8, powers: [8, 9, 6, 4, 10, 3, 2, 5, 7, 1]


***Wykorzystanie dla PRNG***
Ciąg może być wykorzystany jako PRNG, ale jest on cykliczny i deterministyczny, dla danego (g i p). Bezpieczeństow zależy od wyboru g i p, ze względu na deterministyczność warto dodać pewne miksowanie, aby wzmocnić kryptogradiczne bezpieczeństow

### 2. Czy indeks spełnia wymagania klucza algorytmu RSA dla n=p*q

In [28]:
from math import gcd
import random

def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True

def generate_bbs_prime(bits):
    while True:
        p = random.getrandbits(bits)
        if is_prime(p) and p % 4 == 3:
            return p
        
class BBS:
    def __init__(self, bits=32):
        self.bits = bits
        self.p = generate_bbs_prime(bits)
        self.q = generate_bbs_prime(bits)
        self.n = self.p * self.q
        self.state = random.randint(2, self.n - 1)
        while gcd(self.state, self.n) != 1: 
            self.state = random.randint(2, self.n - 1)

    def next_bit(self):
        self.state = pow(self.state, 2, self.n)  
        return self.state % 2  

    def next_bits(self, k):
        return [self.next_bit() for _ in range(k)]
    
    def return_number(self):
        bits_generated = self.next_bits(self.bits)
        return int(''.join(map(str, bits_generated)), 2)
    
def is_valid_RSA_key(p, q, key):
    phi = (p-1) * (q-1)
    return gcd(key, phi) == 1 and key in range(2, phi)

In [64]:
#Generujemy wartości p i q za pomocą BBS_prime
bbs = BBS(32)
p = bbs.return_number()
q = bbs.return_number()

#Sprawdzamy czy indeks jest właściwym kluczem dla n=p*q
index = 411659
print(f"indeks {index} {"jest" if is_valid_RSA_key(p, q, index) else "nie jest"} dobrym kluczem RSA dla p: {p}, q: {q}")

indeks 411659 jest dobrym kluczem RSA dla p: 3080986466, q: 3362754367


### 3. Porównanie efektywności i bezpieczeństwa RSA_0 i RSA_1
RSA_0 nie szyfruje wiadomości blokowo, tylko litera po literze przez co każda litera ma taki sam kod w szyfrogramie. Jest to spory problem, ponieważ korzystając z tej informacji można wnioskować litery oryginalnej wiadomości, RSA_1 koduje litery blokowo, co pozwala uniknąć tego problemu

### 4. Algorytm potęgowania metodą binarną, z pomiarem zmian

In [24]:
def power(a, b, m):
    max_len = len(bin(m)[2:])
    d = 1
    original_d_bin = bin(d)[2:].zfill(max_len)
    k = len(bin(b)[2:]) - 1
    
    for id, i in enumerate(range(k, -1, -1)):
        d = (d * d) % m
        if (b >> i) & 1:
            d = (d * a) % m

        d_bin = bin(d)[2:].zfill(max_len)
        changed_bits = sum([1 for a, b in zip(original_d_bin, d_bin) if a != b])
        changed_bits_percent = (changed_bits / max_len) * 100
        print(f"Iteration: {id+1}:\nResult in binary: {d_bin}\nChanged bits in result: {round(changed_bits_percent, 2)}%")
    return d

In [27]:
message = "a"
message_num = ord(message)
e = 67
p = 29
q = 43
print(power(message_num, e, p*q))


Iteration: 1:
Result in binary: 00001100001
Changed bits in result: 18.18%
Iteration: 2:
Result in binary: 01010101000
Changed bits in result: 45.45%
Iteration: 3:
Result in binary: 01111110010
Changed bits in result: 72.73%
Iteration: 4:
Result in binary: 00000110110
Changed bits in result: 45.45%
Iteration: 5:
Result in binary: 00110100110
Changed bits in result: 54.55%
Iteration: 6:
Result in binary: 01011000000
Changed bits in result: 36.36%
Iteration: 7:
Result in binary: 00110011000
Changed bits in result: 45.45%
408


# 5. Odszyfrowanie szyfrogramu
Wiadomość odczytana z szyfrogramu 92056083186673327707438445422138850995, dla n = p*q = 340277174544854189285703885855116560303 i e = 65537 to ***"Podaj nr albumu"***